# Dataset

In [12]:
from datasets import load_dataset

dataset = load_dataset("ASSERT-KTH/megadiff-sf-synthetic_test_error", split="train")
dataset = dataset.train_test_split(test_size=0.02)

dataset

DatasetDict({
    train: Dataset({
        features: ['diff', 'is_single_chunk', 'is_single_function', 'buggy_function', 'fixed_function', 'short_diff', 'completion', 'generated_test_case', 'generated_error_message', 'prompt', 'answer'],
        num_rows: 63100
    })
    test: Dataset({
        features: ['diff', 'is_single_chunk', 'is_single_function', 'buggy_function', 'fixed_function', 'short_diff', 'completion', 'generated_test_case', 'generated_error_message', 'prompt', 'answer'],
        num_rows: 1288
    })
})

# Tokeniser

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/CodeLlama-7b-Instruct-hf")
tokenizer.add_special_tokens({"pad_token":"<pad>"})

1

In [14]:
def format_texts(examples, begin_inst="[INST]", end_inst="[\\INST]"):
    output_texts = []
    for i in range(len(examples['prompt'])):
        text = f"<s>{begin_inst} {examples['prompt'][i]} {end_inst} {examples['answer'][i]}</s>"
        output_texts.append(text)
    return output_texts

In [16]:
def tokenize_function(batch):
    return tokenizer(format_texts(batch), padding=False, truncation=False)

max_seq_length = 1024

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.filter(lambda x: len(x["input_ids"]) < max_seq_length, batched=False)

print(f"Training dataset size: {len(tokenized_dataset['train'])}")
print(f"Validation dataset size: {len(tokenized_dataset['test'])}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1288 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1288 [00:00<?, ? examples/s]

Training dataset size: 258
Validation dataset size: 330


# Trainer

In [17]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
from accelerate import PartialState

import torch

# FIXME: to enable more than 1 sample per batch, the extra padding token must be set in the model and the embedding layer resized
# the current workaround simply adds a new pad_token so that the eos_token is not ignored during training, since the models needs to learn when to stop
device_string = PartialState().process_index
model = AutoModelForCausalLM.from_pretrained("meta-llama/CodeLlama-7b-Instruct-hf",
                                             torch_dtype=torch.bfloat16,
                                             device_map={'':device_string})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
from trl import DataCollatorForCompletionOnlyLM

response_template_with_context = "[\\INST]"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)

collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

In [19]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

sft_config = SFTConfig(
    output_dir='tmp_trainer',
    learning_rate=5e-4,
    num_train_epochs=1,
    max_seq_length=max_seq_length,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    packing=False,
)

trainer = SFTTrainer(
    model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=sft_config,
    peft_config=peft_config,
    data_collator=collator,
)

/usr/local/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [20]:
for batch in trainer.get_train_dataloader():
    print(batch)
    break

{'input_ids': tensor([[    1,     1,   518, 25580, 29962,   887,   526,   385, 18428,  1824,
         26032,  5780, 29889,  3575,  3414,   338,   304,  2329,   278,  4944,
          6494,  1927,   775, 29889,    13,    13,  1576,  1494,   775,  3743,
           263,  6494,  1927,   740, 29901,    13, 28956,  1645,    13,  1678,
           970,  1780,  1243,  8566,  2631,  2855,  6185,  2631,  4873,  8614,
           580,   426,    13,   308,    13,  4706,  8427,  8614,  2295,   353,
          8427,  8614, 29889,  4381,  8614,   580,    13, 18884,   869,   351,
           929, 29931,   293,  1947, 29898, 21076,  1674,   261, 29889, 29933,
          5371, 29897,    13, 18884,   869, 12007, 29898, 21076,  1674,   261,
         29889, 29933,  5371, 29897,    13, 18884,   869,  3827, 29898, 21076,
          1674,   261, 29889,  1806, 29963, 29897,    13, 18884,   869,  9961,
           345, 29898, 21076,  1674,   261, 29889,  1806, 29963, 29897,    13,
         18884,   869, 12007, 29898, 2

In [21]:
trainer.train()
trainer.save_state()
trainer.save_model(output_dir="codellama-instruct-repair")
tokenizer.save_pretrained(save_directory="codellama-instruct-repair")

Step,Training Loss


('codellama-instruct-repair/tokenizer_config.json',
 'codellama-instruct-repair/special_tokens_map.json',
 'codellama-instruct-repair/tokenizer.json')